In [5]:
import pandas as pd
from imblearn.over_sampling import BorderlineSMOTE, KMeansSMOTE
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
from sklearn.cluster import KMeans
import os

# データ読み込み
normal_path = r'normal2_impute.csv'
abnormal_path = r'anormal2_impute.csv'
df_normal = pd.read_csv(normal_path)
df_abnormal = pd.read_csv(abnormal_path)
imbalance = df_normal.shape[0]/df_abnormal.shape[0]

print(f"imbalance: {imbalance} (正常/故障)")

imbalance: 3.4 (正常/故障)


In [6]:
# フラグ設定
use_borderline_smote = 0
use_kmeans_smote = 1
use_enn = 0
use_tomek_links = 0

# 目標比率(正常:故障=n:1 nの値を記載)
borderline_smote_ratio = 1
kmeans_smote_ratio = 1

# 分析対象外
temp_excluded_features = []

# ラベル付与
df_normal['label'] = 0
df_abnormal['label'] = 1
df_combined = pd.concat([df_abnormal, df_normal], axis=0)

X = df_combined.drop(['label'] + temp_excluded_features, axis=1)
y = df_combined['label']
excluded_features = df_combined[temp_excluded_features]

# Borderline SMOTEの適用
if use_borderline_smote==1:
    target_minority_count = (len(df_normal) // borderline_smote_ratio)
    smote = BorderlineSMOTE(sampling_strategy={1: target_minority_count}, random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    # 新たに追加された行の数を計算
    new_rows_count = len(X_resampled) - len(X)
    # 新たに追加された行に"Gen_BS"を設定
    generated_rows = pd.DataFrame('Gen_BS', index=range(new_rows_count), columns=temp_excluded_features)
    excluded_features_resampled = pd.concat([excluded_features, generated_rows], ignore_index=True)
else:
    X_resampled, y_resampled = X, y
    excluded_features_resampled = excluded_features

# KMeans SMOTEの適用
if use_kmeans_smote==1:
    target_minority_count = (len(df_normal) // kmeans_smote_ratio)-1
    kmeans_smote = KMeansSMOTE(
        sampling_strategy={1: target_minority_count},
        kmeans_estimator=KMeans(n_clusters=4, n_init=20), #def=8, n_init:クラスタ中心をn回初期化し,最小SSEになる中心を選択(def=10)
        cluster_balance_threshold = 0.05, #def=0.1
        n_jobs =-1,
        #random_state=42
    )
    X_resampled, y_resampled = kmeans_smote.fit_resample(X_resampled, y_resampled)
    # 新たに追加された行の数を計算
    new_rows_count = len(X_resampled) - len(excluded_features_resampled)
    # 新たに追加された行に"Gen_KS"を設定
    generated_rows = pd.DataFrame('Gen_KS', index=range(new_rows_count), columns=temp_excluded_features)
    excluded_features_resampled = pd.concat([excluded_features_resampled, generated_rows], ignore_index=True)

# ENNの適用
if use_enn==1:
    enn = EditedNearestNeighbours()
    X_resampled, y_resampled = enn.fit_resample(X_resampled, y_resampled)
    resampled_indices = enn.sample_indices_
    excluded_features_resampled = excluded_features_resampled.iloc[resampled_indices].reset_index(drop=True)

# Tomekリンクの適用
if use_tomek_links==1:
    tomek = TomekLinks()
    X_resampled, y_resampled = tomek.fit_resample(X_resampled, y_resampled)
    resampled_indices = tomek.sample_indices_
    excluded_features_resampled = excluded_features_resampled.iloc[resampled_indices].reset_index(drop=True)

# 基本情報
normal_count = sum(y_resampled == 0)
abnormal_count = sum(y_resampled == 1)
print(f"normal samples: {normal_count}")
print(f"anormal samples: {abnormal_count}")
print(f"imbalance: {normal_count/abnormal_count}")

df_resampled = pd.concat([X_resampled, excluded_features_resampled, y_resampled], axis=1)

# 元のデータの列順序を取得
original_columns = df_combined.columns.tolist()

# 正常異常を分ける
df_normal_resampled = df_resampled[df_resampled['label'] == 0].drop(columns=['label'])
df_abnormal_resampled = df_resampled[df_resampled['label'] == 1].drop(columns=['label'])

# 列順序を元データと同じにする
df_normal_resampled = df_normal_resampled[original_columns[:-1]]  # 'label'列を除く
df_abnormal_resampled = df_abnormal_resampled[original_columns[:-1]]

# 保存
normal_output_path = f"{os.path.splitext(normal_path)[0]}_ou.csv"
abnormal_output_path = f"{os.path.splitext(abnormal_path)[0]}_ou.csv"
df_normal_resampled.to_csv(normal_output_path, index=False)
df_abnormal_resampled.to_csv(abnormal_output_path, index=False)


normal samples: 34
anormal samples: 34
imbalance: 1.0
